In [1]:
import pandas as pd
import numpy as np
import datetime
import os

print(datetime.datetime.now())
print(os.getcwd())

2020-03-11 11:23:33.834664
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/Gianna_DMA_of_stores_20200311


In [2]:
df_DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2],skiprows=1)
df_DMA_zip.columns=['zip_cd','DMA']
df_DMA_zip=df_DMA_zip.drop_duplicates()
df_DMA_list_by_zip=df_DMA_zip.groupby("zip_cd")['DMA'].apply(list).to_frame().reset_index()

In [3]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                           dtype=str,sep="|")
df_store_list=df_store_list[['location_id','zip_cd']]
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_list_output=pd.merge(df_store_list,df_DMA_list_by_zip,on="zip_cd",how="left")
df_store_list_output.head(4)

,location_id,zip_cd,DMA
0,3,30906,[AUGUSTA-AIKEN]
1,30,43402,[TOLEDO]
2,42,45662,[CHARLESTON-HUNTINGTON]
3,45,43616,[TOLEDO]


In [4]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                           dtype=str,sep="|")
df_store_list=df_store_list[['location_id','zip_cd']]
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
print(df_store_list.shape,df_store_list['location_id'].nunique())
df_store_list['zip_cd'].apply(len).unique()

df_store_list_dup=pd.merge(df_store_list,df_DMA_zip,on="zip_cd",how="left")
print(df_store_list_dup.shape)
df_store_list_dup=df_store_list_dup.groupby("DMA")['location_id'].apply(list).to_frame().reset_index()
df_store_list_dup['store_count']=df_store_list_dup['location_id'].apply(len)
print(df_store_list_dup.shape)

(1403, 2) 1403
(1451, 3)
(187, 3)


In [5]:
writer=pd.ExcelWriter("./BL_store_list_by_DMA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_store_list_dup.to_excel(writer,"DMA",index=False)
df_store_list_output.to_excel(writer,"stores",index=False)
writer.save()